In [1]:
datapath = "gs://voter-project-25-235/VM2Uniform--WY--2021-01-13/VM2Uniform--WY--2021-01-13.tab"

In [7]:
wy_data = spark.read.format("csv")\
.option("inferSchema", "true")\
.option("header", "true")\
.option("delimiter", "\t")\
.csv(datapath)

In [15]:
cols = wy_data.columns

In [42]:
from pyspark.sql.functions import col, count, when
def drop_null_columns(df, return_dropped_names = False, threshold = 0):
    """
    Function that drops the columns of a dataframe that contain with only a low number of non-null values.
    This number is specified by `threshold`. `threshold = 0` means columns with entirely null values get dropped.
    Optionally returns a list of column names that get dropped.
    """
    not_null_cnt = df.select([count(when(col(c).isNotNull(), c)).alias(c) for c in df.columns]).collect()[0].asDict()
    to_drop = [k for k, v in not_null_cnt.items() if v <= threshold]
    if return_dropped_names:
        return df.drop(*to_drop), to_drop
    else:
        return df.drop(*to_drop)

In [43]:
wy, d_cols = drop_null_columns(wy_data, True)

In [44]:
len(wy.columns)

416